In [19]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
from math import sqrt

In [30]:
def read(his_file,sim_num,fname): 

# Middle of the vegetation patch  
    jmid=50; imid=45; 
# detect the presence of vegetation patch to know its extents (Currently hardwired
    imin_plant=41 ; imax_plant=50 ; jmin_plant=45 ; jmax_plant=54   
    
    nc = netCDF4.Dataset(his_file)
    mask = nc.variables['mask_rho'][:]

# read x,y
    x = nc.variables['x_rho'][:,:]
    y = nc.variables['y_rho'][:,:] 
    tot_time = nc.variables['ocean_time'] 
    
    imax=len(x[1,:])
    jmax=len(y[:,1])
    
    # read water depth
    depth = nc.variables['h'][:,:]
    theta_s = nc.variables['theta_s'][:]
    theta_b = nc.variables['theta_b'][:]
    depth_c = nc.variables['hc'][:]
    
#Get Stretching function 
    N=60 # Number of vertical levels, kgrid=1 MATLAB code copy 
    Np=N+1 
    ds=1.0/N
    s=np.zeros(Np)
    Csur=np.zeros(Np)
    for i in range(0,Np):
        lev=i
        s[i]=(lev-N)*ds
        Csur[i]=-s[i]**2
    
# Reshape 1D vertical variables to 3D so we can broadcast
    s.shape=(np.size(s),1,1)
    Csur.shape=(np.size(Csur),1,1)
    
# read 3D water level at all time steps
    eta = nc.variables['zeta'] 
    
# calculate the 4D field of z values (vertical coordinate)
    z0=(depth_c*s+depth*Csur)/(depth_c+depth)
    
# Extract the time frame for the last tidal cycle
    t_tidal=[]
    for t in range(1,len(tot_time)):
        if(eta[t,jmid,imid]<0.0 and eta[t+1,jmid,imid]>0.0):
            t_tidal.append(t)

    t_tidal_srt=t_tidal[-2]
    t_tidal_end=t_tidal[-1]
    print "Last tidal cycle starts and ends at this instance" 
    print t_tidal_srt,t_tidal_end   
    t_slice=t_tidal_end-t_tidal_srt+1
                  
# Find water level for all locations at last tidal cycle
#    z=np.zeros(shape=(t_slice,Np,jmax,imax))
#    for t in range(t_tidal_srt,t_tidal_end+1): 
#        eta = nc.variables['zeta'][t,:,:] 
#        z[t-t_tidal_srt,:,:,:] = eta+(eta+depth)*z0
        
    # Calculated integrated TKE over last tidal cycle before and after the patch (3pts.)
    tke_bot_total=0.0
    tke_total=0.0 
    
    for t in range (t_tidal_srt,t_tidal_end+1): 
        tke=nc.variables['tke'][t,:,:,:]
#  
        for j in range(jmin_plant-1,jmax_plant):
            for i in range(imin_plant-1,imax_plant):
                dy=y[j+1,i]-y[j,i]
                dx=x[j,i+1]-x[j,i]
        
                tke_bot_total=tke[1,j,i]+tke_bot_total
                
                for k in range(1,Np): 
                    # change in water level at last time step 
                    # dz = z[t-t_tidal_srt,k,j,i]-z[t-t_tidal_srt,k-1,j,i]          
                    tke_total=tke[k,j,i]+tke_total
                    
    mean_tke=(tke_total)/t_slice
    mean_bot_tke=(tke_bot_total)/t_slice
    print "mean_tke","mean_bot_tke"
    print mean_tke,mean_bot_tke
    
    f.write("%s\t, %s\t  %s\t \n" % (sim_num, mean_tke, mean_bot_tke))


In [31]:
'''Output from vegetation module COAWST Mean chane in planar kinetic energy (before & after the arou
   vegetation patch  '''

fname='tke_inside.txt'
f = open(fname, 'w')
total_sim_list= 16
f.write("Case_no, Mean TKE inside(mts2/sec3), Mean_bottom_TKE inside: \n")

for sim_num in range(1,total_sim_list):
    path1='/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/'
    path2='veg_test_his_'+ str(sim_num)+'_ext' + '.nc'
    url=path1+path2
    print url

    read(url,sim_num,fname) 
     
f.close()

/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_1_ext.nc
Last tidal cycle starts and ends at this instance
180 252
mean_tke mean_bot_tke
0.982942027286 0.0157066565149
/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_2_ext.nc
Last tidal cycle starts and ends at this instance
180 252
mean_tke mean_bot_tke
0.921938664576 0.0152814566145
/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_3_ext.nc
Last tidal cycle starts and ends at this instance
180 252
mean_tke mean_bot_tke
1.00617955475 0.0158136200045
/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_4_ext.nc
Last tidal cycle starts and ends at this instance
180 252
mean_tke mean_bot_tke
0.974924878894 0.0162772074051
/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/veg_test_his_5_ext.nc
Last tidal cycle starts and ends at this instance
180 252
mean_tke mean_bot_tke
0.974957862607 0.0152599081594
/media/gadar/DATADRIVE1/sensitivity_history/run_ext_new/